In [1]:
import semopy
import pandas as pd

In [2]:
consolidated_sem_input_rc = pd.read_csv('SEM_InputData/SEM_Input_Consolidated_5Years.csv')

In [3]:
consolidated_sem_input_rc.columns

Index(['object_id', 'gmted_drainage_density_without_1', 'srtm_filled_dem',
       'GCN250_ARCIII_average', 'Inundation', 'sum',
       'assam_dist_from_major_rivers_updated_3857',
       'strm_filled_slope_degrees', 'ndvi', 'assam_soil_silt',
       'assam_soil_loamysand', 'assam_soil_rocky', 'assam_lith_neogene',
       'assam_lith_paleogene', 'assam_lith_paleozoic', 'assam_lith_quaternary',
       'assam_lith_tertiary', 'assam_lith_undevelopedprecambrian',
       'assam_lith_water', 'landuse_rangeland', 'landuse_vegetation',
       'landuse_water', 'ind_ppp_UNadj', 'aged', 'young', 'percaay',
       'deprived', 'nophone', 'noSanitation', 'nodrinkingWater',
       'totLivestock', 'proximity_hosptial_rd', 'proximity_embankment_rd',
       'proximity_rail_rd', 'proximity_local_rd', 'proximity_arterial_rd',
       'ndbi', 'population_affected', 'crop_area_affected', 'human_lives_lost',
       'roads_dam', 'bridges_dam', 'embankment_total',
       'Count_Roads_Bridges_Embankments', 'Sum_R

In [9]:
consolidated_sem_input_rc_norm = pd.DataFrame()

for column in consolidated_sem_input_rc.columns:
    x = consolidated_sem_input_rc[column]
    if column in ['object_id','district', 'revenue_cr']:
        consolidated_sem_input_rc_norm[column] = x
    else:
        normalized = (x-min(x))/(max(x)-min(x))
        consolidated_sem_input_rc_norm[column] = normalized

In [66]:
print(semopy.efa.explore_pine_model(consolidated_sem_input_rc_norm.fillna(0)))

/home/krishna/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_optics.py:903: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


1 defaultdict(<class 'set'>, {'eta1_1': ['eta2', 'eta7', 'eta9', 'eta6'], 'eta2_1': ['eta5', 'eta13', 'eta3']})
eta2 =~ assam_lith_quaternary + assam_lith_undevelopedprecambrian
eta3 =~ Count_Total + Count_repair_rest_imp + Sum_Total + Count_SOPD + Count_Roads_Bridges_Embankments + Sum_repair_rest_imp + Count_IM + Sum_SOPD + Count_SDRF + Count_Erosion + Sum_Erosion + Sum_Roads_Bridges_Embankments
eta5 =~ young + noSanitation + ind_ppp_UNadj + percaay + nophone + aged + deprived + population_affected
eta6 =~ roads_dam + bridges_dam + embankment_total + population_affected
eta7 =~ gmted_drainage_density_without_1 + srtm_filled_dem + ndbi + proximity_hosptial_rd + ndvi + GCN250_ARCIII_average + proximity_local_rd
eta9 =~ ndvi + strm_filled_slope_degrees + GCN250_ARCIII_average + assam_soil_rocky
eta13 =~ totLivestock + ind_ppp_UNadj
DEFINE(latent) eta1_1 eta2_1
eta1_1 =~ eta2 + eta7 + eta9
eta2_1 =~ eta5 + eta13 + eta3


In [67]:
print(semopy.efa.explore_cfa_model(consolidated_sem_input_rc_norm.fillna(0)))

/home/krishna/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_optics.py:903: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


eta2 =~ assam_lith_quaternary + assam_lith_undevelopedprecambrian
eta3 =~ Count_Total + Count_repair_rest_imp + Sum_Total + Count_SOPD + Count_Roads_Bridges_Embankments + Sum_repair_rest_imp + Count_IM + Sum_SOPD + Count_SDRF + Count_Erosion + Sum_Erosion + Sum_Roads_Bridges_Embankments
eta5 =~ young + noSanitation + ind_ppp_UNadj + percaay + nophone + aged + deprived + population_affected
eta6 =~ roads_dam + bridges_dam + embankment_total + population_affected
eta7 =~ gmted_drainage_density_without_1 + srtm_filled_dem + ndbi + proximity_hosptial_rd + ndvi + GCN250_ARCIII_average + proximity_local_rd
eta9 =~ ndvi + strm_filled_slope_degrees + GCN250_ARCIII_average + assam_soil_rocky
eta13 =~ totLivestock + ind_ppp_UNadj



In [94]:
model_spec = """
# measurement model
flood_impact =~  population_affected + roads_dam + bridges_dam + embankment_total + crop_area_affected
"""

In [95]:
model = semopy.Model(model_spec)

In [96]:
model.fit(consolidated_sem_input_rc_norm.fillna(0),
         obj='MLW',
         solver='de',
        )

/home/krishna/anaconda3/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


SolverResult(fun=0.17445074303248198, success=True, n_it=156, x=array([1.63523789e+00, 1.72522599e+00, 1.51045311e+00, 1.91525389e-01,
       2.80699868e-02, 1.18426704e-03, 3.51493869e-02, 1.19506418e-02,
       1.80303714e-02, 1.20864044e-02]), message='Optimization terminated successfully.', name_method='de', name_obj='MLW')

In [97]:
coeff_df = model.inspect(std_est=True)

In [98]:
coeff_df

,lval,op,rval,Estimate,Est. Std,Std. Err,z-value,p-value
0,population_affected,~,flood_impact,1.000000,0.548619,-,-,-
1,roads_dam,~,flood_impact,1.635238,0.982167,0.202755,8.065101,0.0
2,bridges_dam,~,flood_impact,1.725226,0.816169,0.2198,7.849062,0.0
3,embankment_total,~,flood_impact,1.510453,0.663038,0.217125,6.956609,0.0
4,crop_area_affected,~,flood_impact,0.191525,0.189134,0.078503,2.439722,0.014699
5,flood_impact,~~,flood_impact,0.012086,1.000000,0.003104,3.894398,0.000098
6,population_affected,~~,population_affected,0.028070,0.699017,0.003037,9.242205,0.0
7,roads_dam,~~,roads_dam,0.001184,0.035348,0.001526,0.776139,0.437667
8,embankment_total,~~,embankment_total,0.035149,0.560381,0.003956,8.885505,0.0
9,crop_area_affected,~~,crop_area_affected,0.011951,0.964228,0.001261,9.474481,0.0


In [99]:
stats = semopy.calc_stats(model)
print(stats.T)

                    Value
DoF              5.000000
DoF Baseline    10.000000
chi2            31.401134
chi2 p-value     0.000008
chi2 Baseline  374.262242
CFI              0.927522
GFI              0.916099
AGFI             0.832197
NFI              0.916099
TLI              0.855043
RMSEA            0.171751
AIC             19.651099
BIC             51.580667
LogLik           0.174451


Dof is positive = Over identified model.

In [27]:
data = semopy.examples.political_democracy.get_data()
mod = semopy.examples.political_democracy.get_model()

In [29]:
print(mod)

# measurement model
ind60 =~ x1 + x2 + x3
dem60 =~ y1 + y2 + y3 + y4
dem65 =~ y5 + y6 + y7 + y8
# regressions
dem60 ~ ind60
dem65 ~ ind60 + dem60
# residual correlations
y1 ~~ y5
y2 ~~ y4 + y6
y3 ~~ y7
y4 ~~ y8
y6 ~~ y8


In [4]:
df = gdf[['district_3', 'object_id', 'district_1', 'revenue_cr', 'fld_proneness',
       'demo_vul', 'infr_access', 'flood_damages', 'govt_resp', 'prep_need']]#

In [5]:
df

,district_3,object_id,district_1,revenue_cr,fld_proneness,demo_vul,infr_access,flood_damages,govt_resp,prep_need
0,KOKRAJHAR,101,Kokrajhar,Gossaigaon (Pt),46,151,10,126,164,149
1,KOKRAJHAR,102,Kokrajhar,Bhawraguri,96,27,119,42,136,43
2,KOKRAJHAR,103,Kokrajhar,Dotoma,156,98,157,91,141,100
3,KOKRAJHAR,104,Kokrajhar,Kokrajhar (Pt),29,149,15,143,177,163
4,KOKRAJHAR,105,Kokrajhar,Bagribari (Pt),39,52,32,120,53,69
...,...,...,...,...,...,...,...,...,...,...
175,JORHAT,277,Jorhat,Jorhat East,85,59,142,89,171,103
176,CHARAIDEO,278,Charaideo,Sapekhati,42,58,106,30,33,34
177,CHARAIDEO,279,Charaideo,Sonari,28,97,131,45,32,59
178,MAJULI,280,Majuli,Ujani Majuli,145,31,55,52,154,74


In [11]:
df = pd.melt(df, id_vars=['revenue_cr'], value_vars=['fld_proneness', 'demo_vul', 'infr_access',
                                                'flood_damages', 'govt_resp', 'prep_need'])

In [10]:
import pandas as pd

In [6]:
df.to_csv('sem_latent_variables.csv', index=False)

In [3]:
gdf = gpd.read_file('sem_latent_variables.geojson', driver='GeoJSON')
gdf['geometry'] = gdf.geometry.simplify(tolerance=0.01)

In [32]:
gdf.to_file('sem_latent_variables_simplifiedgeo.geojson',index=False)

In [2]:
import geopandas as gpd